# IVP Euler's method

For each IVP, solve the problem using Euler's method. 
- Plot the solution for $n=320$. 
- For $n=10\cdot2^k$, $k=2,3,\ldots,10$, compute the error as $||u-\hat{u}||_\infty=\max_{0\leq i\leq n} |u_i-\hat{u}(t_i)|$ and at the final time, $|u_n-\hat{u}(t_n)|$. Make a log-log convergence plot, including a reference line for first-order convergence.

In [ ]:
using Plots
using LaTeXStrings
using Printf
using Markdown
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\linear_systems.jl")
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\diff_equations.jl")

In [ ]:
n1 = 320
n = [10*2^k for k in 2:1:10]

function f1(t::Number, u::Number)
    return -2.0 * t * u
end
function f2(t::Number, u::Number)
    return u + t
end
function f3(t::Number, u::Number)
    return t^2/((1+t^3)*u)
end

u01 = 2
u02 = 2
u03 = 1

u1_exact = t -> 2*exp(-t^2)
u2_exact = t -> - 1 - t + 3*exp(t)
u3_exact = t -> sqrt(1 + 2.0/3.0 * log(1 + t^3))

colors = ["#0F2080",  # blu scuro
          "#F5793A",  # arancione
          "#A95AA1",  # viola
          "#85C0F9"]  # azzurro

Errors are estimated from the formula 
$$
|\hat{u}(t_i) - u_i| \le \frac{Ch^p}{L} \left[ e^{L(t_i-a)} - 1
\right] = O(h^p)\,,
$$

As we can see $|\hat{u}(t_i) - u_i| = qh^p$ for some generic $q \in \mathbb{R}$. In order to determine $p$, the order of convergence, we will use the interpolation relation $\log(|\hat{u}(t_i) - u_i|) = \log(q) - p\log(n)$, knowing that $h \simeq \frac{1}{n}$. \
For Euler's method we expect $p=1$. The graphs show the slope of the fits, which will therefore be negative.\
The same occurs for the study of $|\hat{u}(t_n) - u_n|$

E' normale ce gli ultimi due abbiano i grafici degli errori sovrapposti, è dovuto al fatto che l'equazione è esponenziale.
In particolare l'errore dell'ultimo tempo e del massimo coincidono

**(a)** $u' = -2t u$, $\ 0 \le t \le 2$, $\ u(0) = 2$;  

$\quad$ $\hat{u}(t) = 2e^{-t^2}$

In [ ]:
a, b = 0, 2
t = [i for i in a:(b-a)/(n1):b]
u_num = euler_ode(f1, u01, n1, a, b)

fig = plot(figsize=(800, 600),
           xlabel=L"t", ylabel=L"u(t)",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           legend = :topright
          )

plot!(fig, t, u1_exact.(t),
      label = L"\hat{u}(t)",
      color= :navy,
      lw = 4.0,
     ) 

plot!(fig, t, u_num,
      label = L"u_{approx}(t)",
      color= :darkorange,
      linestyle = :dash,
      lw = 1.5,
     ) 


In [ ]:
a, b = 0, 2
#Error section
err_inf = Float64[]
err_last = Float64[]
for nk in n
    t = range(a, b; length=nk+1)
    u_num = euler_ode(f1, u01, nk, a, b)
    u_ex = u1_exact.(t)
    push!(err_inf, maximum(abs.(u_num .- u_ex)))
    push!(err_last, abs(u_num[end] - u_ex[end]))
end

#Fit section
D = hcat(ones(length(n)), log.(n))
c1 = least_sq(D, log.(err_inf))
c2 = least_sq(D, log.(err_last))

#Plot section
format_tick(x) = @sprintf("%.f", x)
format_tick_scientific(x) = @sprintf("%.e", x)

fig = plot(figsize=(800, 600),
           xlabel=L"n", ylabel=L"Errors",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           xscale = :log10, yscale = :log10,
           xticks = n, yticks = [10.0^i for i in 0:-0.25:-16],
           xformatter = format_tick, yformatter = format_tick_scientific,
           legend = :topright
          )

plot!(fig, n, err_inf,
      label = L"||u_i-\hat{u_i}\ ||_\infty",
      color= colors[1],
      marker = :circle,
      lw = 4.0,
     )

plot!(fig, n, exp(c1[1]) .* n .^ c1[2],
    label = L"\mathrm{Fit}\ (\mathrm{slope}=" * @sprintf("%.3f", c[2]) * L")",
    color = colors[2],
    linestyle = :dash,
    lw = 2.0,
     )

plot!(fig, n, err_last,
      label = L"||u_n-\hat{u_n}\ ||",
      color= colors[3],
      marker = :circle,
      lw = 4.0,
    )
plot!(fig, n, exp(c2[1]) .* n .^ c2[2],
    label = L"\mathrm{Fit}\ (\mathrm{slope}=" * @sprintf("%.3f", c2[2]) * L")",
    color = colors[4],
    linestyle = :dash,
    lw = 2.0,
     )

display(fig)

**(b)** $u' = u + t$, $\ 0 \le t \le 1$, $\ u(0) = 2$;  

$\quad$ $\hat{u}(t) = -1-t+3e^t$

In [ ]:
a, b = 0, 1
t = [i for i in a:(b-a)/(n1):b]
u_num = euler_ode(f2, u02, n1, a, b)

fig = plot(figsize=(800, 600),
           xlabel=L"t", ylabel=L"u(t)",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           legend = :bottomright
          )

plot!(fig, t, u2_exact.(t),
      label = L"\hat{u}(t)",
      color= :navy,
      lw = 4.0,
     ) 

plot!(fig, t, u_num,
      label = L"u_{approx}(t)",
      color= :darkorange,
      linestyle = :dash,
      lw = 1.5,
     ) 


In [ ]:
a, b = 0, 1
#Error section
err_inf = Float64[]
err_last = Float64[]
for nk in n
    t = range(a, b; length=nk+1)
    u_num = euler_ode(f2, u02, nk, a, b)
    u_ex = u2_exact.(t)
    push!(err_inf, maximum(abs.(u_num .- u_ex)))
    push!(err_last, abs(u_num[end] - u_ex[end]))
end

#Fit section
D = hcat(ones(length(n)), log.(n))
c1 = least_sq(D, log.(err_inf))
c2 = least_sq(D, log.(err_last))

#Plot section
format_tick(x) = @sprintf("%.f", x)
format_tick_scientific(x) = @sprintf("%.e", x)

fig = plot(figsize=(800, 600),
           xlabel=L"n", ylabel=L"Errors",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           xscale = :log10, yscale = :log10,
           xticks = n, yticks = [10.0^i for i in 0:-0.25:-16],
           xformatter = format_tick, yformatter = format_tick_scientific,
           legend = :topright
          )

plot!(fig, n, err_inf,
      label = L"||u_i-\hat{u_i}\ ||_\infty",
      color= colors[1],
      marker = :circle,
      lw = 4.0,
     )

plot!(fig, n, exp(c1[1]) .* n .^ c1[2],
    label = L"\mathrm{Fit}\ (\mathrm{slope}=" * @sprintf("%.3f", c1[2]) * L")",
    color = colors[2],
    linestyle = :dash,
    lw = 2.0,
     )
plot!(fig, n, err_last,
      label = L"||u_n-\hat{u_n}\ ||",
      color= colors[3],
      marker = :circle,
      lw = 4.0,
     )
plot!(fig, n, exp(c2[1]) .* n .^ c2[2],
    label = L"\mathrm{Fit}\ (\mathrm{slope}=" * @sprintf("%.3f", c2[2]) * L")",
    color = colors[4],
    linestyle = :dash,
    lw = 2.0,
     )

display(fig)

**(c)** $(1+t^3)uu' = t^2$, $\ 0 \le t \le 3$, $\ u(0) =1$; 

$\quad$ $\hat{u}(t) = [1+(2/3)\ln (1+t^3)]^{1/2}$

In [ ]:
a, b = 0, 3
t = [i for i in a:(b-a)/(n1):b]
u_num = euler_ode(f3, u03, n1, a, b)

fig = plot(figsize=(800, 600),
           xlabel=L"t", ylabel=L"u(t)",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           legend = :bottomright,
          )

plot!(fig, t, u3_exact.(t),
      label = L"\hat{u}(t)",
      color= :navy,
      lw = 4.0,
     ) 

plot!(fig, t, u_num,
      label = L"u_{approx}(t)",
      color= :darkorange,
      linestyle = :dash,
      lw = 1.5,
     ) 


In [ ]:
a, b = 0, 3
#Error section
err_inf = Float64[]
err_last = Float64[]
for nk in n
    t = range(a, b; length=nk+1)
    u_num = euler_ode(f2, u02, nk, a, b)
    u_ex = u2_exact.(t)
    push!(err_inf, maximum(abs.(u_num .- u_ex)))
    push!(err_last, abs(u_num[end] - u_ex[end]))
end

#Fit section
D = hcat(ones(length(n)), log.(n))
c1 = least_sq(D, log.(err_inf))
c2 = least_sq(D, log.(err_last))

#Plot section
format_tick(x) = @sprintf("%.f", x)
format_tick_scientific(x) = @sprintf("%.e", x)

fig = plot(figsize=(800, 600),
           xlabel=L"n", ylabel=L"Errors",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           xscale = :log10, yscale = :log10,
           xticks = n, yticks = [10.0^i for i in 1:-0.25:-16],
           xformatter = format_tick, yformatter = format_tick_scientific,
           legend = :topright
          )

plot!(fig, n, err_inf,
      label = L"||u_i-\hat{u_i}\ ||_\infty",
      color= colors[1],
      marker = :circle,
      lw = 4.0,
     )

plot!(fig, n, exp(c1[1]) .* n .^ c1[2],
    label = L"\mathrm{Fit}\ (\mathrm{slope}=" * @sprintf("%.3f", c1[2]) * L")",
    color = colors[2],
    linestyle = :dash,
    lw = 2.0,
     )
plot!(fig, n, err_last,
      label = L"||u_n-\hat{u_n}\ ||",
      color= colors[3],
      marker = :circle,
      lw = 4.0,
     )
plot!(fig, n, exp(c2[1]) .* n .^ c2[2],
    label = L"\mathrm{Fit}\ (\mathrm{slope}=" * @sprintf("%.3f", c2[2]) * L")",
    color = colors[4],
    linestyle = :dash,
    lw = 2.0,
     )

display(fig)